In [ ]:
import kpi
import resdk
import pandas as pd
import datetime
from ggplot import *
from kpi.settings.local import DEPLOYS
DEPLOYS.pop('qa2')  # remove QA2 from the stats
from local import IGNORE_USERNAMES

In [ ]:
pd.set_option('display.expand_frame_repr', False)  # prints data to full width
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200

Prepare variables and functions

In [ ]:
last_30_days = datetime.date.today() - datetime.timedelta(days=30)  # used for calculating remaining demo days on APP
last_24_hrs = datetime.date.today() - datetime.timedelta(days=1)  # used for calculating daily stats
last_7_days = datetime.date.today() - datetime.timedelta(days=7)  # used for calculating weekly stats
deploys = [DEPLOYS[i]['url'] for i in DEPLOYS]  # used for checking if the deploy is running
exclude_usernames = IGNORE_USERNAMES

# Prepare data

### Fetch data

In [ ]:
raw_samples = kpi.data.platform.sample
raw_data = kpi.data.platform.data
raw_processes = kpi.data.platform.process
raw_users = kpi.data.platform.user

### Prepare data for users

In [ ]:
users = []
for user in raw_users:
    xy = pd.DataFrame({
        'id': [user['id']],
        'first_name': [user['first_name']],
        'last_name': [user['last_name']],
        'username': [user['username']],
        'email': [user['email']],
        'phone_number': [user['phone_number']],
        'company': [user['company']],
        'department': [user['department']],
        'lab': [user['lab']],
        'job_title': [user['job_title']],
        'location': [user['location']],
        'created': [user['created']],
        'date_joined': [user['date_joined']],
        'modified': [user['modified']],
        'deploy': [user['deploy']],
    }, columns = ['deploy', 'id', 'first_name', 'last_name', 'username', 'email',
                  'phone_number', 'company', 'department', 'lab', 'job_title',
                  'location', 'created', 'date_joined', 'modified'])
    users.append(xy)

users = pd.concat(users)

users['demo_ramaining_days'] = last_30_days - users.date_joined

### Prepare data for samples

In [ ]:
samples = []
for sample in raw_samples:
    tags = [i.split(':')[-1] for i in sample['tags']]
    xy = pd.DataFrame({
        'id': sample['id'],
        'username': sample['contributor']['username'],
        'first_name': sample['contributor']['first_name'],
        'last_name': sample['contributor']['last_name'],
        'deploy': sample['deploy'],
        'created': sample['created'],
        'modified': sample['modified'],
        'name': sample['name'],
        'tags': tags
    },
        columns=['id', 'username', 'first_name', 'last_name', 'deploy', 'created', 'modified', 'name', 'tags']
    )

    samples.append(xy)

samples = pd.concat(samples)

### Prepare data for data

In [ ]:
data = []
for datum in raw_data:
    tags = ', '.join(datum['tags'])
    
    xy = pd.DataFrame({
        'id': [datum['id']],
        'username': [datum['contributor']['username']],
        'first_name': [datum['contributor']['first_name']],
        'last_name': [datum['contributor']['last_name']],
        'created': [datum['created']],
        'deploy': [datum['deploy']],
        'started': [datum['started']],
        'finished': [datum['finished']],
        'modified': [datum['modified']],
        'name': [datum['name']],
        'process_name': [datum['process_name']],
        'process_error': [datum['process_error']],
        'process_info': [datum['process_info']],
        'process_warning': [datum['process_warning']],
        'process_progress': [datum['process_progress']],
        'status': [datum['status']],
        'tags': tags
    },
        columns=['id', 'username', 'first_name', 'last_name', 'created', 'deploy',
            'started', 'finished', 'modified', 'name', 'process_name', 'process_error',
            'process_info', 'process_warning', 'process_progress', 'status', 'tags']
    )
    
    data.append(xy)

data = pd.concat(data)

### Prepare data for processes

In [ ]:
processes = []
for process in raw_processes:
    xy = pd.DataFrame({
        'username': [process['contributor']['username']],
        'first_name': [process['contributor']['first_name']],
        'last_name': [process['contributor']['last_name']],
        'created': [process['created']],
        'modified': [process['modified']],
        'deploy': [process['deploy']],
        'process_name': [process['name']],
        'type': [process['version']],
        'version': [process['version']]
    },
        columns=['username', 'first_name', 'last_name', 'deploy', 'created', 'modified', 'process_name', 'type', 'version']
    )
    processes.append(xy)

processes = pd.concat(processes)

### Are all the deploys up and running?

In [ ]:
for deploy in deploys:
    try:
        res = resdk.Resolwe(url=deploy)
        res.user.count()
        print("[OK] Connected to {dpl}".format(dpl=deploy))
    except:
        print("[FAIL] Connected to {dpl}".format(dpl=deploy))

### Which users are in the on-boarding period, and how many days? Have them ordered by time so that it helps the management.

In [ ]:
onboard_users = users[users.username.isin(exclude_usernames) == False]
onboard_users = onboard_users[onboard_users.date_joined >= last_30_days][['demo_ramaining_days', 'deploy', 'date_joined', 'username', 'first_name',
                                          'last_name', 'email']].sort_index(axis=0, ascending=False)
onboard_users

### Have users exceeded the allocated sample number?

In [ ]:
samples[['username', 'tags']] \
    .groupby(['username', 'tags']) \
    .size() \
    .reset_index(name='num_samples') \
    .sort_values(['username',], ascending = True)

### What were crucial events on all deploys and communities in the last 24  hours (new samples, new users, failed samples)?

##### Users joined in the last 24 hours

In [ ]:
users[users.date_joined >= last_24_hrs]

##### Samples created in the last 24 hours.

In [ ]:
samples[samples.created >= last_24_hrs].sort_values(by=['created'], ascending=False)

##### Samples modified in the last 24 hours.

In [ ]:
samples[samples.modified >= last_24_hrs].sort_values(by=['modified'], ascending=False)

### Are there any active users or analyses that would suffer from sudden loss of service (e.g. at re-deploy).

What is the status of data objects?

In [ ]:
pd.DataFrame(data.status.value_counts()).rename({'OK': 'Done', 'ER': 'Error', 'RE': 'Resolving', 'WT': 'Waiting', 'PR': 'Processing'})

Objects that are waiting or being processed.

In [ ]:
data[data.status.isin(['WT', 'PR', 'RE'])].sort_values(by=['created', 'deploy'], ascending=False)

### What users have not generated any new objects in an arbitrary period of time (e.g. last week/month)?

Which users have not created any objects in the past 7 days. Sorted by date joined.

In [ ]:
list_of_users = pd.DataFrame(users.username.unique(), columns=['username'])
list_of_users = users[['username', 'first_name', 'last_name', 'date_joined', 'deploy']]
obj_count_users = pd.DataFrame(data[data.created >= last_7_days].username.value_counts())
obj_count_users.index.name = 'nm'
obj_count_users.reset_index(inplace=True)
obj_count_users.columns = ['username', 'count']
week_obj_count = pd.merge(list_of_users, obj_count_users, on='username', how='outer')
week_obj_count = week_obj_count[week_obj_count['count'].isna()] \
    .sort_values(by=['deploy', 'date_joined', 'username'], ascending=False)[['username', 'first_name', 'last_name', 'deploy', 'count']]
week_obj_count[week_obj_count['username'].isin(exclude_usernames) == False]

Which users have not created any objects in the past 30 days. Sorted by date joined.

In [ ]:
list_of_users = pd.DataFrame(users.username.unique(), columns=['username'])
list_of_users = users[['username', 'first_name', 'last_name', 'date_joined', 'deploy']]
obj_count_users = pd.DataFrame(data[data.created >= last_30_days].username.value_counts())
obj_count_users.index.name = 'nm'
obj_count_users.reset_index(inplace=True)
obj_count_users.columns = ['username', 'count']
week_obj_count = pd.merge(list_of_users, obj_count_users, on='username', how='outer')
week_obj_count = week_obj_count[week_obj_count['count'].isna()] \
    .sort_values(by=['deploy', 'date_joined', 'username'], ascending=False)[['username', 'first_name', 'last_name', 'deploy', 'count']]
week_obj_count[week_obj_count['username'].isin(exclude_usernames) == False]

### Which processes are being run by which user?

In [ ]:
ct = data[data.username.isin(exclude_usernames) == False]
pd.crosstab(ct['username'], ct['process_name'])

### How many new users or new samples in an arbitrary period of time per selected deploy/community?

### Average and median number of samples per user.

In [ ]:
mean_smp_by_usr = samples[samples.username.isin(exclude_usernames) == False]
mean_smp_by_usr = mean_smp_by_usr.groupby(['username', 'deploy']).size().to_frame(name='num_samples').reset_index()
mean_smp_by_usr_stats = pd.DataFrame({
    'x': [mean_smp_by_usr.num_samples.mean(), mean_smp_by_usr.num_samples.median()],
    'y': [8, 7],
    'stat': ['mean', 'median'] 
})

ggplot(aes(x='num_samples'), data=mean_smp_by_usr) + \
    theme_bw() + \
    ggtitle('Samples per user') + \
    geom_histogram(binwidth=5) + \
    geom_vline(x=mean_smp_by_usr_stats.x)

### Average and median number of samples in the trial period per user.

No mean/median number of samples for trial users yet. When there are more users, a histogram or a statistic will be more appropriate.

In [ ]:
mean_smp_by_onboardusr = samples[samples.username.isin(exclude_usernames) == False]
mean_smp_by_onboardusr = mean_smp_by_onboardusr[mean_smp_by_onboardusr.username.isin(onboard_users.username) == True]
mean_smp_by_onboardusr = mean_smp_by_onboardusr.groupby(['username', 'deploy']).size().to_frame(name='num_samples').reset_index()
mean_smp_by_onboardusr

### Average and median size of a sample in selected community.

Mean number of samples per user by deploy-tag combination.

In [ ]:
smp_by_com = samples[samples.username.isin(exclude_usernames) == False]
smp_by_com.groupby(['deploy', 'tags', 'username']).size().groupby(['deploy', 'tags']).mean()

Median number of samples per user by deploy-tag combination.

In [ ]:
smp_by_com.groupby(['deploy', 'tags', 'username']).size().groupby(['deploy', 'tags']).median()

### How many active users within an arbitrary period of time, where active is defined as “on the app” in that period.

Users that have created or modified samples or data objects in the last 30 days.

In [ ]:
active_usr_30d = samples[samples.created >= last_30_days]
active_usr_30d = active_usr_30d[active_usr_30d.username.isin(exclude_usernames) == False]
active_usr_30d.groupby(['username']).size().reset_index(name='num_samples')

Users that have created or modified data objects in the last 30 days.

In [ ]:
data_usr_30d = data[(data.created >= last_30_days) | (data.modified >= last_30_days)]
data_usr_30d = data_usr_30d[data_usr_30d.username.isin(exclude_usernames) == False]
data_usr_30d.groupby(['username']).size().reset_index(name='num_data')

### What users have not accessed the software in an arbitrary period of time (e.g. last week/month)?